In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")

In [2]:
import os
import json
import numpy as np
import pandas as pd
from explorer.utils import load_states_from_dir

/Users/patrickjohnchia/Desktop/Projects/negotiation/.venv/lib/python3.8/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [9]:
# log_dir_bs = "/Users/patrickjohnchia/Desktop/Projects/negotiation/.logs/buysell"
log_dir = "/Users/patrickjohnchia/Desktop/Projects/negotiation/.logs/buysell"
game_states = load_states_from_dir(log_dir) #+ load_states_from_dir(log_dir) 

THERE ARE 5 log files
5 Loaded Successfully


In [10]:
def get_from_summary(key, game_state, default=None):
    if key in game_state.game_state[-1]['summary']:
        return game_state.game_state[-1]['summary'][key]
    else:
        return default

def compute_game_summary(game_states):
    print(game_states[0].game_state[-1]['summary'].keys())
    game_name = np.array([g.__class__.__name__ for g in game_states])[:, None]
    models = np.array([[ p.model for p in g.players] for g in game_states])
    outcomes = np.array([get_from_summary('player_outcome', g) for g in game_states])
    valuations = np.array([get_from_summary('player_valuation', g, default=[None,None])  for g in game_states])
    initial_resources = np.array([get_from_summary('initial_resources', g) for g in game_states])
    final_resources = np.array([get_from_summary('final_resources', g) for g in game_states]), 
    resources_delta = (final_resources-initial_resources)[0]
        
    resources_delta = np.array([v.value(r) if v else r.value() for r,v in zip(resources_delta.reshape(-1,), valuations.reshape(-1))])
    resources_delta = resources_delta.reshape(-1, 2)

    df = np.concatenate((game_name, models, outcomes, resources_delta), axis=1)
    df = pd.DataFrame(df, columns=['game_name','model_1','model_2','outcome_1','outcome_2', 'resource_delta_1', 'resource_delta_2'])
    # for column in ['outcome_1','outcome_2', 'resource_delta_1', 'resource_delta_2']:
    #     print(df[column])
    #     df[column] = pd.to_numeric(df[column])
    

    return df
      
df = compute_game_summary(game_states)
df

dict_keys(['player_goals', 'initial_resources', 'proposed_trade', 'player_valuation', 'final_response', 'final_resources', 'player_outcome'])


,game_name,model_1,model_2,outcome_1,outcome_2,resource_delta_1,resource_delta_2
0,BuySellGame,gpt-4-1106-preview,gpt-4-1106-preview,-9.0,-1.0,-9.0,-1.0
1,BuySellGame,gpt-4-1106-preview,gpt-4-1106-preview,0.0,0.0,0.0,0.0
2,BuySellGame,gpt-4-1106-preview,gpt-4-1106-preview,-10.0,0.0,-10.0,0.0
3,BuySellGame,gpt-4-1106-preview,gpt-4-1106-preview,-10.0,0.0,-10.0,0.0
4,BuySellGame,gpt-4-1106-preview,gpt-4-1106-preview,-11.0,1.0,-11.0,1.0
